In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import os

In [20]:

def loaddatasets(num_train):
    all_data = pickle.load(open('all_data'))
    all_label = pickle.load(open('all_label'))
    train_data = all_data[:num_train].astype(np.float32)
    train_label = all_label[:num_train, -5:].astype(np.int32)
    test_data = all_data[num_train:].astype(np.float32)
    test_label =  all_label[num_train:, -5].astype(np.int32)
    return train_data, train_label, test_data, test_label


def acc(predictions,label):
    return  (100.0 * np.sum((np.array(predictions).reshape(1,-1)+0.5).astype(np.int32)==label.reshape(1,-1)) / float(predictions.shape[0]*5))



In [21]:
train_data, train_label, test_data, test_label = loaddatasets(900000)

In [17]:
np.sum(train_label)/float(train_label.shape[0])

0.49944888888888889

In [22]:

#train_data, train_label, test_data, test_label = pickle.load(open('smalldata'))
print (train_data.shape, test_data.shape)

#print(train_data)
#print(test_label)
print ('DATA LOADED')
batch_size = 128
beta = 0.000

h1_nodes = 128
h2_nodes = 512
h3_nodes = 512

graph = tf.Graph()

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, 128))
    tf_train_label = tf.placeholder(tf.float32, shape=(batch_size, 5))
    tf_test_dataset = tf.constant(train_data)

    # H1
    h1_w = tf.Variable(tf.truncated_normal([128, h1_nodes]))
    h1_b = tf.Variable(tf.truncated_normal([h1_nodes]))
    h1 = tf.nn.sigmoid(tf.matmul(tf_train_dataset, h1_w) + h1_b)

    # H2
#     h2_w = tf.Variable(tf.truncated_normal([h1_nodes, h2_nodes]))
#     h2_b = tf.Variable(tf.truncated_normal([h2_nodes]))
#     h2 = tf.nn.relu(tf.matmul(h1, h2_w) + h2_b)

    #H3
#     h3_w = tf.Variable(tf.truncated_normal([h2_nodes, h3_nodes]))
#     h3_b = tf.Variable(tf.truncated_normal([h3_nodes]))
#     h3 = tf.nn.relu(tf.matmul(h2, h3_w) + h3_b)

    # last

    w = tf.Variable(tf.truncated_normal([h1_nodes, 5]))
    b = tf.Variable(tf.truncated_normal([5]))

    logits = tf.matmul(h1, w)+b

    loss= tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits,tf_train_label) + beta * tf.nn.l2_loss(w))

    optimizer= tf.train.AdamOptimizer(0.001).minimize(loss)

    prediction = tf.nn.sigmoid(logits)
    test_relu1= tf.nn.sigmoid(tf.matmul(tf_test_dataset,h1_w)+h1_b)
#     test_relu2= tf.nn.relu(tf.matmul(test_relu1,h2_w)+h2_b)
#     test_relu3= tf.nn.relu(tf.matmul(test_relu2,h3_w)+h3_b)
    test_prediction= tf.nn.sigmoid(tf.matmul(test_relu1,w)+b)


(900000, 128) (100000, 128)
DATA LOADED


In [23]:

numstep = 10000
with tf.Session(graph=graph) as ses:
    tf.initialize_all_variables().run()
    print ('TF INIT')
    for step in range(numstep):
        offset = (step*batch_size) % (train_label.shape[0]-batch_size)
        batch_data=train_data[offset:offset+batch_size,:]
        batch_label=train_label[offset:offset+batch_size,:]

        feed_dict= {tf_train_dataset:batch_data, tf_train_label:batch_label,}
        _,l,pp = ses.run([optimizer,loss,prediction],feed_dict=feed_dict)
        if step % 500==0:
        
            print ("batch ACC %d :%f"%(step,acc(pp,batch_label)))
            tt=test_prediction.eval()
            #print (train_label.reshape(-1,1)[:10])
            #print ((np.array(tt[:10])+0.5).astype(np.int32))
            print ("Test ACC %.1f"%acc(tt,train_label))


TF INIT
batch ACC 0 :51.562500
Test ACC 50.0
batch ACC 500 :50.781250
Test ACC 50.0
batch ACC 1000 :49.843750
Test ACC 50.0
batch ACC 1500 :47.812500
Test ACC 50.0
batch ACC 2000 :47.968750
Test ACC 50.0
batch ACC 2500 :48.593750
Test ACC 50.0
batch ACC 3000 :50.156250
Test ACC 50.0
batch ACC 3500 :50.312500
Test ACC 50.0
batch ACC 4000 :53.281250
Test ACC 50.0
batch ACC 4500 :49.843750
Test ACC 50.0
batch ACC 5000 :49.687500
Test ACC 50.0
batch ACC 5500 :48.125000
Test ACC 50.0
batch ACC 6000 :49.218750
Test ACC 50.0
batch ACC 6500 :53.593750
Test ACC 50.0
batch ACC 7000 :52.187500
Test ACC 50.0
batch ACC 7500 :50.625000
Test ACC 50.0
batch ACC 8000 :51.093750
Test ACC 50.0
batch ACC 8500 :52.968750
Test ACC 50.1
batch ACC 9000 :48.437500
Test ACC 50.0
batch ACC 9500 :47.343750
Test ACC 50.0


In [23]:
(np.array([0.2])+.5).astype(np.int32)

array([0], dtype=int32)

In [14]:
w.run()

AttributeError: 'Variable' object has no attribute 'run'

In [4]:
from sklearn import svm

In [10]:
clf = svm.LinearSVC(verbose=3)

In [11]:
f=clf.fit(train_data[:100000,:],train_label[:100000])

[LibLinear]

In [13]:
f.score(test_data[:10000,:],test_label[:10000])

0.51060000000000005